# LINE Messaging API

# Global vars

In [16]:
PLANT_NAME = "DEFAULT NAME" # we need to find a way to keep this in the memory.
#Since the code is always running and only sends messages at some intervals, it shouldn't be that hard to do

## Broadcast Message

In [17]:
import requests
import os
from typing import Dict, Any, Optional, List

class PlantitaBot:
    def __init__(self, channel_access_token: str):
        """Initialize the Plantita LINE bot

        Args:
            channel_access_token (str): Your LINE channel access token
        """
        self.channel_access_token = channel_access_token
        self.headers = {
            'Authorization': f'Bearer {channel_access_token}',
            'Content-Type': 'application/json'
        }
        self.base_url = 'https://api.line.me/v2/bot'

    def get_followers(self) -> List[str]:
        """Get list of user IDs who are friends with your LINE Official Account

        Returns:
            List[str]: List of user IDs
        """
        user_ids = []
        next_cursor = None

        while True:
            endpoint = f'{self.base_url}/followers/ids'
            if next_cursor:
                endpoint += f'?start={next_cursor}'

            response = requests.get(endpoint, headers=self.headers)
            data = response.json()

            if 'userIds' in data:
                user_ids.extend(data['userIds'])

            if 'next' not in data:
                break

            next_cursor = data['next']

        return user_ids

    def broadcast_message(self, message: str) -> Dict[str, Any]:
        """Broadcast a message to all friends of your LINE Official Account

        Args:
            message (str): The message to broadcast

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/broadcast'

        data = {
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()




    def broadcast_plant_status(self,
                               temperature: float,
                               humidity: float,
                               light_level: float,
                               status: str = "normal") -> Dict[str, Any]:
        """Broadcast plant status to all friends

        Args:
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.broadcast_message(message)

    def send_message(self, user_id: str, message: str) -> Dict[str, Any]:
        """Send a message to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            message (str): The message to send

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/push'

        data = {
            'to': user_id,
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()

    def send_plant_status(self,
                          user_id: str,
                          temperature: float,
                          humidity: float,
                          light_level: float,
                          status: str = "normal") -> Dict[str, Any]:
        """Send plant status to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.send_message(user_id, message)

## Example Usage of Broadcast

## Example Usage of Direct Message
**Still Not Working** Need to setup Webhook Server.

In [18]:
# Initialize the bot
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")



## GROQ Implementation

# Install Dependency

# Demonstration

In [19]:
import os

from groq import Groq

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '''You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.
Current readings for Monstera Deliciosa:
        temperature=27.5,
        humidity=55,
        light_level=40,

Ideal conditions for this plant:
- Temperature: 20°C to 25°C
- Humidity: 60% to 80%
- Light Level: 50% to 80%

Please analyze these conditions and provide:
1. A caring, conversational assessment of the plant's current environment
2. Specific recommendations for improvement if needed
3. Any potential risks to the plant's health based on these conditions
4. A simple action plan for the plant owner

Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants.
Limit your response to fit a notification on a messaging app, focus only on what the user needed to do and keep it to three to 5 sentences''',
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Hey there, sweetie! I'm checking in on your beautiful Monstera Deliciosa, and I think I can offer some helpful advice.

Temperature-wise, your plant is doing just fine at 27.5°C. It's a bit on the warmer side, but it can handle it. However, I do think it would thrive even more if it was around 22-23°C.

Humidity-wise, your plant is a bit on the dry side at 55%. I think it would love it if you could aim for 60-80% humidity. You can try using a humidifier or placing it on a tray filled with water and pebbles.

Light-wise, your plant is getting some great light at 40%, but I think it would really flourish if it could get 50-80% light. You can try moving it to a spot that gets a bit more direct light, but not too much direct sunlight.

Some potential risks to watch out for are fungal diseases and pests, which can be more common in dry environments. But don't worry, just keep an eye out for any changes and take action if needed.

Here's a simple action plan for you:

* Check your plant's te

# Integrate sensor information with Groq

# Simulate Arduino data on PC if we forgot to carry the Arduino

# Actual arduino data

In [20]:
#!pip install bleak


In [21]:

# FOR TESTING PURPOSES
import asyncio
from bleak import BleakClient
import nest_asyncio

# # Apply nested asyncio compatibility for Jupyter or similar environments
# nest_asyncio.apply()

# DEVICE_ADDRESS = "19:9F:19:C0:C2:42"  # Replace with your device's address

# # UUIDs for the characteristics
# TEMPERATURE_UUID = "2A6E"
# HUMIDITY_UUID = "2A6F"
# PRESSURE_UUID = "2A6D"
# SOIL_MOISTURE_UUID = "2A70"

# # Callback for disconnection
# def handle_disconnection(client):
#     print("Disconnected from central.")

# async def main():
#     client = BleakClient(DEVICE_ADDRESS, timeout=30.0)

#     # Assign the disconnection callback
#     client.set_disconnected_callback(handle_disconnection)

#     try:
#         await client.connect()
#         if not client.is_connected:
#             print("Failed to connect to device.")
#             return

#         print("Connected to device.")
#         while True:
#             try:
#                 # Read characteristic values
#                 temperature = await client.read_gatt_char(TEMPERATURE_UUID)
#                 humidity = await client.read_gatt_char(HUMIDITY_UUID)
#                 pressure = await client.read_gatt_char(PRESSURE_UUID)
#                 soil_moisture = await client.read_gatt_char(SOIL_MOISTURE_UUID)

#                 # Decode data (assuming IEEE 754 single-precision float, little-endian)
#                 import struct
#                 temperature = struct.unpack('<f', temperature)[0]
#                 humidity = struct.unpack('<f', humidity)[0]
#                 pressure = struct.unpack('<f', pressure)[0]
#                 soil_moisture = struct.unpack('<f', soil_moisture)[0]

#                 # Print the decoded values
#                 print(f"Temperature: {temperature:.2f} °C")
#                 print(f"Humidity: {humidity:.2f} %")
#                 print(f"Soil Moisture: {soil_moisture:.2f} %")
#                 print("----------------------------")

#                 await asyncio.sleep(1)
#             except KeyboardInterrupt:
#                 print("Stopped by user.")
#                 break
#     except Exception as e:
#         print(f"An error occurred: {e}")
#     finally:
#         if client.is_connected:
#             await client.disconnect()
#         print("Program ended.")

# # Use the event loop
# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())


In [22]:
import os
from groq import Groq



# Step 1: Generate the message from Groq
def generate_plant_status_message(temperature, humidity, pressure, moisture, plant_name):
    """
    Generates a plant care message using the Groq API.
    """
    user_message = (
        f"You are Plantita, an expert plant care advisor.\n"
        f"Current readings for {plant_name}:\n"
        f"    temperature={temperature}°C,\n"
        f"    humidity={humidity}%,\n"
        f"    moisture={moisture}%,\n\n"
        
        "Ideal conditions:\n"
        "- Temperature: [Fill in with the values according to the {plant_name}]\n"
        "- Humidity: [Fill in with the values according to the {plant_name}]\n"
        "- Moisture: [Fill in with the values according to the {plant_name}]\n\n"
        "Provide a friendly message for the user."
        "If Temperature, Humidity, or Moisture are not within the ideal range, include '|<ALERT>|\n' in the message."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    return response.choices[0].message.content


# Step 2: Broadcast or send the generated message
# This is the wrapper function that combines the two steps
def send_plant_status_to_followers(temperature, humidity, pressure, moisture, plant_name):
    client = Groq(
    api_key="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",
    )

    bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")

    message = generate_plant_status_message(temperature, humidity, pressure, moisture,  plant_name)
    if "|<ALERT>|" in message:
        # Delete the alert tag and send the message
        message = message.replace("|<ALERT>|", "")
        bot.broadcast_message(message)
    else:
        # Don't send the message if everything is normal
        pass




In [23]:
import asyncio
import serial
import time
import struct
from datetime import datetime, timedelta
from bleak import BleakClient
import nest_asyncio

# Apply nested asyncio compatibility for environments like Jupyter
nest_asyncio.apply()

# Constants for the messaging interval
MESSAGE_INTERVAL = timedelta(minutes=1)
PLANT_NAME = "Eustoma russellianum"  # Example plant name
next_message_time = datetime.now()


# BLE settings
DEVICE_ADDRESS = "19:9F:19:C0:C2:42"  # Replace with your BLE device address
TEMPERATURE_UUID = "2A6E"
HUMIDITY_UUID = "2A6F"
PRESSURE_UUID = "2A6D"
SOIL_MOISTURE_UUID = "2A70"

time.sleep(2)  # Allow time for the connection to establish


In [24]:

async def read_ble_sensor_data(client):
    """Reads sensor data from the BLE device."""
    try:
        temperature = await client.read_gatt_char(TEMPERATURE_UUID)
        humidity = await client.read_gatt_char(HUMIDITY_UUID)
        pressure = await client.read_gatt_char(PRESSURE_UUID)
        soil_moisture = await client.read_gatt_char(SOIL_MOISTURE_UUID)

        # Decode data using struct (assuming IEEE 754 single-precision float, little-endian)
        temperature = struct.unpack('<f', temperature)[0]
        humidity = struct.unpack('<f', humidity)[0]
        pressure = struct.unpack('<f', pressure)[0]
        soil_moisture = struct.unpack('<f', soil_moisture)[0]

        return {
            "temperature": temperature,
            "humidity": humidity,
            "pressure": pressure,
            "soil_moisture": soil_moisture
        }
    except Exception as e:
        print(f"Error reading BLE data: {e}")
        return None

def parse_sensor_data(data):
    """Parses the serial sensor data block."""
    try:
        lines = data.strip().split('\n')
        sensor_data = {}
        for line in lines:
            key, value = line.split(':')
            sensor_data[key.strip()] = float(value.strip())
        return sensor_data
    except Exception as e:
        print(f"Error parsing sensor data: {e}")
        return None



In [25]:
async def monitor_plant():
    """Main monitoring loop using BLE data."""
    print("Collecting data and monitoring plant status...")
    global next_message_time

    async with BleakClient(DEVICE_ADDRESS, timeout=30.0) as client:
        if not client.is_connected:
            print("Failed to connect to BLE device.")
            return

        print("Connected to BLE device.")
        try:
            while True:
                # Read BLE data
                ble_data = await read_ble_sensor_data(client)
                if ble_data:
                    print("BLE Data Processed:", ble_data)

                    # Check if it's time to send a message
                    if datetime.now() >= next_message_time:
                        send_plant_status_to_followers(
                            ble_data["temperature"],
                            ble_data["humidity"],
                            ble_data["pressure"],
                            ble_data["soil_moisture"],
                            PLANT_NAME
                        )
                        # Set the next message time
                        next_message_time = datetime.now() + MESSAGE_INTERVAL

                # Wait before the next iteration
                await asyncio.sleep(1)

        except KeyboardInterrupt:
            print("Monitoring stopped.")
        finally:
            print("Disconnected from BLE device.")

# Start the asyncio event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(monitor_plant())

BleakDeviceNotFoundError: Device with address 15944C71-777D-BEE8-9741-8A8D1D671386 was not found

Sometimes it is necessary to reset the Arduino when the connection is lost.